In [ ]:
import numpy as np
import torch
from grid2vec import *
import json
import pandas as pd
from utils import Timer, copy_file
from joblib import Parallel, delayed
from traj2grid import Traj2Grid
import traj_dist.distance as tdist
from logging import raiseExceptions
from parameters import *
import numpy as np
import time

timer = Timer()

In [ ]:
str_grid2idx = json.load(open("data/str_grid2idx_300.json"))
grid2idx = {eval(c): str_grid2idx[c] for c in list(str_grid2idx)}
print(len(grid2idx))

In [ ]:
max_lon = 0
max_lat = 0
min_lon = 999
min_lat = 999
timer.tik()
for i in range(1, 31):
    df = pd.read_csv("data/full/gps_201611"+str(i).zfill(2),
                     names=["name", "order_id", "time", "lon", "lat"],
                     usecols=["lon", "lat"])
    max_lon = df['lon'].max() if df['lon'].max() > max_lon else max_lon
    min_lon = df['lon'].min() if df['lon'].min() < min_lon else min_lon
    max_lat = df['lat'].max() if df['lat'].max() > max_lat else max_lat
    min_lat = df['lat'].min() if df['lat'].min() < min_lat else min_lat
    timer.tok(str(i).zfill(2))
    print(max_lon, min_lon, max_lat, min_lat)

In [ ]:
with open('data/str_grid2idx_800.json') as f:
    str_grid2idx = json.load(f)
    f.close()
grid2idx = {eval(c): str_grid2idx[c] for c in list(str_grid2idx)}

dataset = GridEmbeddingDataset(grid2idx, window_size=20, neg_rate=100)
model = Grid2Vec(len(dataset), 256)
cp = torch.load('model/checkpoint_256_299_0_0.694.pth')
model.load_state_dict(cp['model'])
weight2 = np.load("model/grid_embedding_256_0.8.npy")
weight3 = np.load("model/grid_embedding_384_0.73.npy")
print(evaluate_grid2vec(model.input_embedding(), dataset, test_num=500))
print(evaluate_grid2vec(weight2, dataset, test_num=500))
print(evaluate_grid2vec(weight3, dataset, test_num=500))
